In [24]:
import random
import time
import os
from itertools import tee, islice, chain
import numpy as np
from tqdm import tqdm
from sklearn.metrics import mean_absolute_percentage_error

In [19]:
# Function to calculate the total distance of a route
def total_distance(route, distance_lookup):
    total_dist = 0.0
    num_cities = len(route)

    for i in range(num_cities):
        total_dist += distance_lookup[(route[i], route[(i + 1) % num_cities])]

    return total_dist

In [3]:
# Function to initialize a population with random routes
def initialize_population(pop_size, num_cities):
    population = []
    for _ in range(pop_size):
        route = list(range(num_cities))
        random.shuffle(route)
        population.append(route)
    return population

# Tournament selection function
def tournament_selection(population, distances, k):
    selected = random.sample(population, k)
    return min(selected, key=lambda x: total_distance(x, distances))

# PMX crossover function
def pmx_crossover(parent1, parent2):
    size = len(parent1)
    a, b = random.sample(range(size), 2)
    if a > b:
        a, b = b, a

    child = parent1[a:b+1]
    child_set = set(child)

    for i in range(size):
        if i < a or i > b:
            gene = parent2[i]
            while gene in child_set:
                idx = parent2.index(gene)
                gene = parent2[(idx + 1) % size]
            child.append(gene)
            child_set.add(gene)

    return child

# Inversion mutation function
def inversion_mutation(route):
    a, b = random.sample(range(len(route)), 2)
    if a > b:
        a, b = b, a
    route[a:b+1] = reversed(route[a:b+1])
    return route

# Exchange mutation function
def exchange_mutation(route):
    a, b = random.sample(range(len(route)), 2)
    route[a], route[b] = route[b], route[a]
    return route

# Function to generate a new population and evaluate their routes
def generate_population_and_evaluate(population, distance_lookup, tournament_size):
    new_population = []

    for _ in range(len(population) // 2):
        parent1 = tournament_selection(population, distance_lookup, tournament_size)
        parent2 = tournament_selection(population, distance_lookup, tournament_size)

        if random.random() < crossover_prob:
            child1 = pmx_crossover(parent1, parent2)
            child2 = pmx_crossover(parent2, parent1)
        else:
            child1, child2 = parent1[:], parent2[:]

        if random.random() < inversion_prob:
            child1 = inversion_mutation(child1)
        if random.random() < inversion_prob:
            child2 = inversion_mutation(child2)
        if random.random() < exchange_prob:
            child1 = exchange_mutation(child1)
        if random.random() < exchange_prob:
            child2 = exchange_mutation(child2)

        fitness_child1 = total_distance(child1, distance_lookup)
        fitness_child2 = total_distance(child2, distance_lookup)

        if fitness_child1 < fitness_child2:
            new_population.append(child1)
            new_population.append(parent2)
        else:
            new_population.append(child2)
            new_population.append(parent1)

    return new_population

In [4]:
# Genetic algorithm with improved calculation
def genetic_algorithm_with_elitism(pop_size, tournament_size, crossover_prob, inversion_prob, exchange_prob, num_generations, elitism_ratio, distance_lookup):
    population = initialize_population(pop_size, len(distance_lookup))
    elitism_count = int(elitism_ratio * pop_size)

    for generation in range(num_generations):
        new_population = generate_population_and_evaluate(population, distance_lookup, tournament_size)
        new_population.sort(key=lambda x: total_distance(x, distance_lookup))
        
        # Preserve the best individuals from the current population
        elite_individuals = new_population[:elitism_count]
        
        # Generate the rest of the population through genetic operations
        non_elite_population = new_population[elitism_count:]
        offspring_population = generate_population_and_evaluate(non_elite_population, distance_lookup, tournament_size)
        
        # Combine elite and offspring populations to form the next generation
        population = elite_individuals + offspring_population

    best_route = min(population, key=lambda x: total_distance(x, distance_lookup))
    best_distance = total_distance(best_route, distance_lookup)

    return best_route, best_distance


In [16]:
# Parameters
X = np.load('X_20x20.npy')
Y = np.load('Y_20x20.npy')
border = 60000
X_train = X[:border]
Y_train = Y[:border]
X_test = X[border:]
Y_test = Y[border:]

In [39]:
pop_size = 100
tournament_size = 3
crossover_prob = 0.85
inversion_prob = 0.15
exchange_prob = 0.15
num_generations = 500
elitism_ratio = 0.05
random.seed(1)
ld = []
lp = []
for i in tqdm(range(X_test.shape[0])):
    a = X_test[i]
    route = Y_test[i]
    distance = total_distance(route, a)
    route, total_dist = genetic_algorithm_with_elitism(pop_size, tournament_size, crossover_prob, inversion_prob, 
                                                       exchange_prob, num_generations, elitism_ratio, distance_lookup=a)
    ld.append(distance)
    lp.append(total_dist)    
Y_true = np.array(ld)
Y_predict = np.array(lp)


100%|██████████| 3000/3000 [08:51<00:00,  5.65it/s]


In [40]:
mean_absolute_percentage_error(Y_true, Y_predict)

0.04827466058419728

In [41]:
# Сколько случаев действительно плохого прогноза
sum(((Y_predict - Y_true) / Y_true) > 0.2)

np.int64(0)

In [42]:
%timeit genetic_algorithm_with_elitism(pop_size, tournament_size, crossover_prob, inversion_prob, \
    exchange_prob, num_generations, elitism_ratio, distance_lookup=a)

179 ms ± 2.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
